In [9]:
import os
import re
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from tensorboard.backend.event_processing import event_accumulator
import pickle

In [10]:
def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    # assert all(
    #     s in ea.Tags()["scalars"] for s in scalars
    # ), "some scalars were not found in the event accumulator"
    res = {}
    for s in scalars:
        try:
            res[s] = pd.DataFrame(ea.Scalars(s))
        except:
            continue
    return res

In [3]:
# scalars = [
#     "val/acc_adv_attack",
#     "val/balanced_acc_adv_attack",
#     "val/acc_task",
#     "val/balanced_acc_task",
#     "train/zero_ratio_adv",
#     "train/zero_ratio_task",
#     "val/acc_task_debiased",
#     "val/acc_protected",
#     "val/balanced_acc_task_debiased",
#     "val/balanced_acc_protected"
# ]
# old suffixes for adv attack: adv_attack_unbiased, adv_attack_biased
scalars = [
    "val/acc_task_debiased",
    "val/acc_task",
    "val/balanced_acc_task_debiased",
    "val/balanced_acc_task",
    "val/acc_adv_attack",
    "val/balanced_acc_adv_attack",
    "val/acc_adv_attack_debiased",
    "val/balanced_acc_adv_attack_debiased",
    # "val/acc_adv_attack_unbiased",
    # "val/balanced_acc_adv_attack_unbiased",
    # "val/acc_adv_attack_biased",
    # "val/balanced_acc_adv_attack_biased"
]

In [22]:
[re.sub(r"(?<=seed)[\d+]", "{}", n) for n in os.listdir(folder)]

['only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-p

In [23]:
folder = "/share/home/lukash/pan16/bertl4/logs_only_adv_attack"
experiment_names = set([re.sub(r"(?<=seed)[\d+]", "{}", n) for n in os.listdir(folder)]) # remove seed suffix
results = {}
for n in experiment_names:
    results[n] = []
    for i in range(5):
        n_seed = n.format(i, i)
        filepath = os.path.join(folder, n_seed)
        df = parse_tensorboard(filepath, scalars)
        results[n].append(df)

In [ ]:
# with open("results.pkl", "wb") as f:
#     pickle.dump(results, f)

In [ ]:
# with open("results.pkl", "rb") as f:
#     results = pickle.load(f)

In [24]:
scalar_stats = {}
for s in scalars:
    scalar_stats[s] = {}
    for k,v in results.items():
        try:
            scalar = np.array([x[s]["value"].max() for x in v if s in x])
        except KeyError:
            continue
        scalar_stats[s][k] = np.array([scalar.mean(), scalar.std()])

/tmp/lukash/ipykernel_66637/1661473811.py:9: RuntimeWarning: Mean of empty slice.
  scalar_stats[s][k] = np.array([scalar.mean(), scalar.std()])
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [25]:
list(scalar_stats.keys())

['val/acc_task_debiased',
 'val/acc_task',
 'val/balanced_acc_task_debiased',
 'val/balanced_acc_task',
 'val/acc_adv_attack',
 'val/balanced_acc_adv_attack',
 'val/acc_adv_attack_debiased',
 'val/balanced_acc_adv_attack_debiased']

In [26]:
experiment_names

{'only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}',
 'only_adv_attack_task-diff_pruning_0.1-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}'}

In [27]:
for n in sorted(experiment_names):
    scalars_to_print = ['val/acc_task', 'val/balanced_acc_adv_attack', 'val/acc_task_debiased', 'val/balanced_acc_adv_attack_debiased']
    format_stats = lambda arr: f"{arr[0]:.3f} +- {arr[1]:.3f}"
    res = {k:[_v for _k,_v in v.items() if _k == n][0] for k,v in scalar_stats.items() if k in scalars_to_print}
    res_format = {k:format_stats(v) for k,v in res.items() if not np.isnan(v).any()}
    print(n)
    print(res_format)
    print("\n")

only_adv_attack_task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}
{'val/balanced_acc_adv_attack': '0.463 +- 0.018'}


only_adv_attack_task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}
{'val/balanced_acc_adv_attack': '0.497 +- 0.011'}


only_adv_attack_task-diff_pruning_0.1-bert_uncased_L-4_H-256_A-4-64-2e-05-seed{}-64-0.0001-weighted_loss_prot-prot_idx_1-seed{}
{'val/balanced_acc_adv_attack': '0.495 +- 0.007'}




In [ ]:
n = 'adverserial-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-weighted_loss_prot-gender'
# scalars_to_print = ['val/acc_task', 'val/balanced_acc_adv_attack']
scalars_to_print = ['val/acc_task_debiased', 'val/balanced_acc_adv_attack_debiased']
format_stats = lambda arr: f"{{${arr[0]*100:.1f} \pm {arr[1]*100:.1f}$}}"

res = {k:[_v for _k,_v in v.items() if _k == n][0] for k,v in scalar_stats.items()}
s = " & ".join([format_stats(res[k]) for k in scalars_to_print])
print(s)